# Neural Networks

In [1]:
import pandas as pd

In [2]:
y_train = pd.read_feather('data/processed/y_train.feather')['voted_up'].to_numpy()
X_train = pd.read_feather(f'data/processed/X_train_bigram.feather').to_numpy()

In [3]:
y_test = pd.read_feather('data/processed/y_test.feather')['voted_up'].to_numpy()
X_test = pd.read_feather(f'data/processed/X_test_bigram.feather').to_numpy()

## Baseline Neural Network

In [5]:
model = Sequential()

# hidden layers
model.add(LSTM(500, input_dim=8000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))

# output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_data=(X_test, y_test))
model.evaluate(X_test, y_test)

In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras import Sequential

In [8]:
model = Sequential()

# hidden layers
model.add(Dense(500, input_dim=8000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))

# output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_data=(X_test, y_test))
model.evaluate(X_test, y_test)

Epoch 1/3
1828/1828 [==============================] - 61s 32ms/step - loss: 0.3386 - accuracy: 0.8589 - val_loss: 0.2210 - val_accuracy: 0.9097
Epoch 2/3
1828/1828 [==============================] - 53s 29ms/step - loss: 0.1787 - accuracy: 0.9328 - val_loss: 0.2159 - val_accuracy: 0.9190
Epoch 3/3
457/457 [==============================] - 4s 9ms/step - loss: 0.2229 - accuracy: 0.9180


[0.2228986769914627, 0.9179890751838684]

0.9179890751838684

Top performing base model: 0.914090 test accuracy

## Hyperparameter Tuning

In [5]:
from talos import Scan

In [6]:
def dense_network(x_train, y_train, x_val, y_val, params):
    model = Sequential()

    # hidden layers
    model.add(Dense(params['dense'], input_dim=8000, activation=params['activation1']))
    model.add(Dropout(params['dropout']))
    model.add(Dense(params['dense']*2, activation=params['activation1']))
    model.add(Dropout(params['dropout']))
    model.add(Dense(params['dense']*0.5, activation=params['activation1']))
    model.add(Dropout(params['dropout']))
    model.add(Dense(params['dense']*0.75, activation=params['activation1']))
    # output layer
    model.add(Dense(1, activation=params['activation2']))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    out = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=5,
                    verbose=0)

    return out, model

In [7]:
params = {'dropout': [0.25, 0.5, 0.75], 
          'dense': [10, 50, 100, 500], 
          'activation1': ['relu', 'elu'], 
          'activation2': ['sigmoid', 'tanh']}

In [8]:
results = Scan(X_train, y_train, params=params, model=dense_network, experiment_name='grid')
results.best_model(metric='accuracy')

100%|██████████| 48/48 [1:14:46<00:00, 93.47s/it] 


In [9]:
pd.read_csv('grid/022421165029.csv').sort_values('val_accuracy', ascending=False)

,round_epochs,loss,accuracy,val_loss,val_accuracy,activation1,activation2,dense,dropout
10,5,0.038711,0.988078,0.414048,0.911247,relu,sigmoid,500,0.50
22,5,0.107399,0.980114,0.465812,0.910050,relu,tanh,500,0.50
41,5,0.338274,0.902475,0.373536,0.909252,elu,tanh,50,0.75
11,5,0.139434,0.951653,0.270693,0.908853,relu,sigmoid,500,0.75
4,5,0.113932,0.960887,0.254306,0.908682,relu,sigmoid,50,0.50
23,5,0.192981,0.952728,0.379954,0.907712,relu,tanh,500,0.75
9,5,0.018463,0.994503,0.530807,0.907655,relu,sigmoid,500,0.25
44,5,0.345217,0.908875,0.448334,0.907484,elu,tanh,100,0.75
18,5,0.041759,0.990765,0.592345,0.907085,relu,tanh,100,0.25
26,5,0.318125,0.877287,0.241634,0.906743,elu,sigmoid,10,0.75


## Final Model

In [5]:
model = Sequential()

# hidden layers
model.add(Dense(500, input_dim=8000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(125, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(250, activation='relu'))

# output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
1645/1645 [==============================] - 53s 31ms/step - loss: 0.3298 - accuracy: 0.8628 - val_loss: 0.2276 - val_accuracy: 0.9082
Epoch 2/10
1645/1645 [==============================] - 49s 30ms/step - loss: 0.1735 - accuracy: 0.9360 - val_loss: 0.2266 - val_accuracy: 0.9099
Epoch 3/10
1645/1645 [==============================] - 49s 30ms/step - loss: 0.1182 - accuracy: 0.9577 - val_loss: 0.2352 - val_accuracy: 0.9082
Epoch 4/10
1645/1645 [==============================] - 49s 30ms/step - loss: 0.0719 - accuracy: 0.9752 - val_loss: 0.2849 - val_accuracy: 0.9060
Epoch 5/10
1645/1645 [==============================] - 50s 30ms/step - loss: 0.0404 - accuracy: 0.9877 - val_loss: 0.3743 - val_accuracy: 0.90660s - loss: 0.0404 - ac
Epoch 6/10
1645/1645 [==============================] - 51s 31ms/step - loss: 0.0297 - accuracy: 0.9902 - val_loss: 0.3590 - val_accuracy: 0.9061
Epoch 7/10
1645/1645 [==============================] - 49s 30ms/step - loss: 0.0236 - accuracy: 0.992

In [7]:
model.evaluate(X_test, y_test)

457/457 [==============================] - 4s 9ms/step - loss: 0.4588 - accuracy: 0.9112


[0.4588494598865509, 0.9112175107002258]

In [8]:
model.save('final_model/model.h5')
model.save_weights('final_model/model_weights.h5')